# Apply Ranking Recovery Algorithms

In [1]:
import pandas as pd
import seaborn as sns

from fairpair import *

In [2]:
# create a FairPairGraph with disadvantaged minority
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
# run 100 iterations of ProbKnockout sampling
sampler = ProbKnockoutSampling(G, warn=False)
sampler.apply(iter=100, k=1)
# apply davidScore for ranking recovery
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore) # by default, apply rankCentrality method
ranker._print_with_score(ranking) # sorted by rank score

,node,orig score,rank score
0,7,0.580013,5.106121
1,6,0.668207,4.641327
2,11,0.510724,3.342614
3,14,0.473869,2.323386
4,0,0.581508,-2.426958
5,10,0.439113,-2.426958
6,2,0.118991,-2.710858
7,8,0.503975,-2.710858
8,5,0.689543,-5.137816


## Metrics

In [4]:
# score-based metrics
print("Majority NDCG (by score):", NDCG(G.majority, ranking)) # NDCG is only calculated as a "in-group" comparison because it is normalized
print("Minority NDCG (by score):", NDCG(G.minority, ranking))
print("Majority MSE (by score):", MSE(G.majority, ranking))
print("Minority MSE (by score):", MSE(G.minority, ranking))

Majority NDCG (by score): 0.9489629421738941
Minority NDCG (by score): 0.9999999999999998
Majority MSE (by score): 0.15325552858152244
Minority MSE (by score): 0.06525816989775333


In [5]:
# rank-based metrics
print("Majority mean error (by rank):", rank_mean_error(G, ranking, G.majority))
print("Minority mean error (by rank):", rank_mean_error(G, ranking, G.minority))
print("Majority MSE (by rank):", rank_MSE(G, ranking, G.majority))
print("Minority MSE (by rank):", rank_MSE(G, ranking, G.minority))
print("Majority Spearman's rho:", spearmanr(G, ranking, G.majority))
print("Minority Spearman's rho:", spearmanr(G, ranking, G.minority))

Majority mean error (by rank): 1.5
Minority mean error (by rank): 6.333333333333333
Majority MSE (by rank): 23.833333333333332
Minority MSE (by rank): 41.0
Majority Spearman's rho: PearsonRResult(statistic=0.21955220803673203, pvalue=0.6759632443742956)
Minority Spearman's rho: PearsonRResult(statistic=0.9607689228305226, pvalue=0.1789123750220673)
